# Mettabook

## Setup

In [61]:
# Optional: confirm you're set up to connect to the services used in this notebook
#    If the command does not run, run `./install.sh` from your terminal

!metta status --components=core,system,aws,wandb --non-interactive

Component | Installed  | Connected As              | Expected             | Status
----------------------------------------------------------------------------------
core     | Yes        | -                         | -                    |OK
system   | No         | -                         | -                    |NOT INSTALLED
aws      | Yes        | 767406518141              | 751442549699         |WRONG ACCOUNT
wandb    | Yes        | metta-research            | metta-research       |OK
----------------------------------------------------------------------------------
Some components are not installed. Run 'metta install' to set them up.
Components not installed: system
To fix: metta install system


In [62]:
%load_ext autoreload
%autoreload 2
import os

import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from experiments.notebooks.utils.metrics import fetch_metrics
from experiments.notebooks.utils.monitoring import monitor_training_statuses
from experiments.notebooks.utils.replays import show_replay
from experiments.notebooks.utils.training import launch_training
from datetime import datetime
from metta.common.wandb.wandb_runs import find_training_runs
from metta.rl.trainer_config import TrainerConfig, CheckpointConfig, TorchProfilerConfig, SimulationConfig
from datetime import datetime
from metta.rl.trainer_config import TrainerConfig, CheckpointConfig, TorchProfilerConfig, SimulationConfig
from experiments.notebooks.utils.training import launch_training
from datetime import datetime

%matplotlib inline
plt.style.use("default")

print("Setup complete! Auto-reload enabled.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Setup complete! Auto-reload enabled.


## Set Up MettaGrid Environment

In [63]:
from metta.mettagrid.config import builder

env_cfg = builder.arena(num_agents=2)
# map_cfg = builder.maps.arena.basic()


print(env_cfg.model_dump())

ValidationError: 1 validation error for PyActionsConfig
noop
  Input should be a valid dictionary or instance of PyActionConfig [type=model_type, input_value=PyActionConfig(enabled=Tr..., consumed_resources={}), input_type=PyActionConfig]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type

## Configure Trainer

In [ ]:
# Example: Launch training



run_name = f"{os.environ.get('USER')}.training-run.{datetime.now().strftime('%Y-%m-%d_%H-%M')}"
print(f"Launching training with run name: {run_name}...")

trainer_cfg = TrainerConfig(
    num_workers=6,
    profiler=TorchProfilerConfig(
        profile_dir="s3://softmax-public/profiles/${run}"
    ),
    checkpoint=CheckpointConfig(
        checkpoint_dir="s3://softmax-public/checkpoints/${run}"
    ),
    simulation=SimulationConfig(
        replay_dir="s3://softmax-public/replays/${run}"
    ),
    curriculum="env/mettagrid/arena/basic",
)

print(trainer_cfg)

Launching training with run name: daveey.training-run.2025-08-08_21-25...
total_timesteps=10000000000 ppo=PPOConfig(clip_coef=0.1, ent_coef=0.0021, gae_lambda=0.916, gamma=0.977, max_grad_norm=0.5, vf_clip_coef=0.1, vf_coef=0.44, l2_reg_loss_coef=0, l2_init_loss_coef=0, norm_adv=True, clip_vloss=True, target_kl=None) optimizer=OptimizerConfig(type='adam', learning_rate=0.000457, beta1=0.9, beta2=0.999, eps=1e-12, weight_decay=0) prioritized_experience_replay=PrioritizedExperienceReplayConfig(prio_alpha=0.0, prio_beta0=0.6) vtrace=VTraceConfig(vtrace_rho_clip=1.0, vtrace_c_clip=1.0) zero_copy=True require_contiguous_env_ids=False verbose=True batch_size=524288 minibatch_size=16384 bptt_horizon=64 update_epochs=1 scale_batches_by_world_size=False cpu_offload=False compile=False compile_mode='reduce-overhead' profiler=TorchProfilerConfig(interval_epochs=10000, profile_dir='s3://softmax-public/profiles/${run}') forward_pass_minibatch_target_size=4096 async_factor=2 hyperparameter_scheduler

## Launch Training

In [ ]:
print(trainer_cfg)
# # View `launch_training` function for all options
# result = launch_training(
#     run_name=run_name,
#     curriculum="env/mettagrid/arena/basic",
#     wandb_tags=[f"{os.environ.get('USER')}-arena-experiment"],
#     additional_args=["--skip-git-check"],
# )

Launching training with run name: daveey.training-run.2025-08-08_21-12...
total_timesteps=50000000000 ppo=PPOConfig(clip_coef=0.1, ent_coef=0.0021, gae_lambda=0.916, gamma=0.977, max_grad_norm=0.5, vf_clip_coef=0.1, vf_coef=0.44, l2_reg_loss_coef=0, l2_init_loss_coef=0, norm_adv=True, clip_vloss=True, target_kl=None) optimizer=OptimizerConfig(type='adam', learning_rate=0.0004573146765703167, beta1=0.9, beta2=0.999, eps=1e-12, weight_decay=0) prioritized_experience_replay=PrioritizedExperienceReplayConfig(prio_alpha=0.0, prio_beta0=0.6) vtrace=VTraceConfig(vtrace_rho_clip=1.0, vtrace_c_clip=1.0) zero_copy=True require_contiguous_env_ids=False verbose=True batch_size=524288 minibatch_size=16384 bptt_horizon=64 update_epochs=1 scale_batches_by_world_size=False cpu_offload=False compile=False compile_mode='reduce-overhead' profiler=TorchProfilerConfig(interval_epochs=10000, profile_dir='s3://softmax-public/profiles/${run}') forward_pass_minibatch_target_size=4096 async_factor=2 hyperparame

## Monitor Training Jobs

In [ ]:
# Monitor Training
run_names = ["daveey.navigation.low_reward.baseline.2", "daveey.navigation.low_reward.baseline.07-18"]

# Optional: instead, find all runs that meet some criteria
# run_names = find_training_runs(
#     # wandb_tags=["low_reward"],
#     # state="finished",
#     author=os.getenv("USER"),
#     limit=5,
# )

df = monitor_training_statuses(run_names, show_metrics=["_step", "overview/reward"])

## Fetch Metrics

In [ ]:
metrics_dfs = fetch_metrics(run_names, samples=500)

## Analyze Metrics

In [ ]:
# Plot overview metrics for all fetched runs
if not metrics_dfs:
    print("No metrics data available. Please fetch metrics first.")
else:
    print(f"Plotting metrics for {len(metrics_dfs)} runs")

    # Find common metrics across all runs
    all_columns = set()
    for _, df in metrics_dfs.items():
        all_columns.update(df.columns)

    columns = ["overview/reward", "losses/explained_variance"]
    plot_cols = []

    for col in all_columns:
        if col not in columns:
            continue
        # Check if this column exists in at least one run with numeric data
        has_numeric_data = False
        for df in metrics_dfs.values():
            if col in df.columns and pd.api.types.is_numeric_dtype(df[col]) and df[col].nunique() > 1:
                has_numeric_data = True
                break
        if has_numeric_data:
            plot_cols.append(col)

    if not plot_cols:
        print("No plottable metrics found")
    else:
        # Calculate grid dimensions
        n_metrics = len(plot_cols)
        n_cols = min(3, n_metrics)  # Max 3 columns
        n_rows = (n_metrics + n_cols - 1) // n_cols

        # Create subplots
        fig = make_subplots(
            rows=n_rows,
            cols=n_cols,
            subplot_titles=[col.replace("overview/", "").replace("_", " ") for col in plot_cols],
            vertical_spacing=0.08,
            horizontal_spacing=0.1,
        )

        # Color palette for different runs
        colors = ["blue", "red", "green", "orange", "purple", "brown", "pink", "gray", "olive", "cyan"]

        # Add traces for each metric and each run
        for idx, col in enumerate(plot_cols):
            row = (idx // n_cols) + 1
            col_idx = (idx % n_cols) + 1

            # Plot each run for this metric
            for run_idx, (run_name, df) in enumerate(metrics_dfs.items()):
                if col in df.columns and "_step" in df.columns:
                    color = colors[run_idx % len(colors)]

                    # Only show legend on first subplot to avoid clutter
                    show_legend = idx == 0

                    fig.add_trace(
                        go.Scatter(
                            x=df["_step"],
                            y=df[col],
                            mode="lines",
                            name=run_name,
                            line=dict(color=color, width=2),
                            showlegend=show_legend,
                            legendgroup=run_name,  # Group all traces from same run
                        ),
                        row=row,
                        col=col_idx,
                    )

        # Update layout
        runs_text = "run" if len(metrics_dfs) == 1 else "runs"
        fig.update_layout(
            height=250 * n_rows,
            showlegend=True,
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        )

        # Update x-axes labels for bottom row
        for col_idx in range(1, min(n_cols, n_metrics) + 1):
            fig.update_xaxes(title_text="Steps", row=n_rows, col=col_idx)

        fig.show()

## View Replays

Display replay viewer for a specific run:

In [ ]:
# Show available replays
# replays = get_available_replays("daveey.lp.16x4.bptt8")

# Show the last replay for a run
show_replay("daveey.lp.16x4.bptt8", step="last", width=1000, height=600)